In [1]:
import sys
import re 
import numpy as np 
import pandas as pd 
from pprint import pprint

In [2]:
import gensim, spacy, logging, warnings

In [3]:
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

In [4]:
from nltk.corpus import stopwords
stop_words = stopwords.words ('english')

In [5]:
stop_words.extend(['from', 'to', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'na', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come', 'br', 'sub', 'mech','non'])

In [6]:
%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [7]:
df = pd.read_csv('/Users/maximascientia/Downloads/mimic3d.csv')
print(df.shape)
df.head()

(58976, 28)


,hadm_id,gender,age,LOSdays,admit_type,admit_location,AdmitDiagnosis,insurance,religion,marital_status,...,NumMicroLabs,NumNotes,NumOutput,NumRx,NumProcEvents,NumTransfers,NumChartEvents,ExpiredHospital,TotalNumInteract,LOSgroupNum
0,100001,F,35,6.17,EMERGENCY,CLINIC REFERRAL/PREMATURE,DIABETIC KETOACIDOSIS,Private,PROTESTANT QUAKER,DIVORCED,...,0.65,0.05,5.19,14.91,1.13,0.65,398.70,0,493.89,1
1,100003,M,59,4.04,EMERGENCY,EMERGENCY ROOM ADMIT,UPPER GI BLEED,Private,NOT SPECIFIED,SINGLE,...,1.24,1.59,5.45,7.18,0.99,1.24,373.02,0,465.71,1
2,100006,F,48,12.04,EMERGENCY,EMERGENCY ROOM ADMIT,COPD FLARE,Private,NOT SPECIFIED,SINGLE,...,0.33,0.15,4.15,6.23,0.00,0.33,286.21,0,344.00,3
3,100007,F,73,7.29,EMERGENCY,EMERGENCY ROOM ADMIT,BOWEL OBSTRUCTION,Private,JEWISH,MARRIED,...,0.69,0.17,9.05,11.52,0.00,0.96,526.06,0,603.05,1
4,100009,M,60,4.88,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,CORONARY ARTERY DISEASE,Private,CATHOLIC,MARRIED,...,0.61,0.34,16.19,25.00,2.87,2.05,554.92,0,679.84,1


In [8]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  

# Convert to list
df['AdmitProcedure'] = df['AdmitProcedure'].apply(str)
data = df.AdmitProcedure.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1])

[['na']]


In [9]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [10]:
!python3 -m spacy download en_core_web_sm
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load("en_core_web_sm")
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

     |████████████████████████████████| 13.9 MB 2.4 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [11]:
data_ready = process_words(data_words)

In [28]:
# Create Dictionary
id2word = corpora.Dictionary(data_ready)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

pprint(lda_model.print_topics())

[(0,
  '0.197*"nec" + 0.114*"art" + 0.112*"cor" + 0.064*"vaccination" + '
  '0.061*"cath" + 0.043*"venous" + 0.040*"bypass" + 0.034*"int" + 0.032*"mam" '
  '+ 0.030*"tracheostomy"'),
 (1,
  '0.044*"vent" + 0.039*"invasive" + 0.036*"valve" + 0.031*"percu" + '
  '0.031*"brain" + 0.030*"excision" + 0.026*"endosc" + 0.024*"close" + '
  '0.023*"phototherapy" + 0.022*"opn"'),
 (2,
  '0.089*"tube" + 0.083*"insert" + 0.080*"endotracheal" + 0.068*"tis" + '
  '0.057*"circumcision" + 0.024*"hemodialysis" + 0.019*"transfusion" + '
  '0.018*"spinal" + 0.018*"tap" + 0.018*"meninge"')]


In [29]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_ready, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/Users/maximascientia/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload



Coherence Score:  0.6821145710942993


In [30]:
# Perplexity: measures the performance of the model. If it low, it is good.
print('\nPerplexity: ', lda_model.log_perplexity(corpus))


Perplexity:  -4.842962551329531


In [31]:
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

/Users/maximascientia/opt/anaconda3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.265781  0.280607       1        1  39.838552
1     -0.116686 -0.364423       2        1  31.018567
2      0.382467  0.083816       3        1  29.142881, topic_info=               Term         Freq        Total Category  logprob  loglift
6               nec  7248.000000  7248.000000  Default  30.0000  30.0000
9               art  4186.000000  4186.000000  Default  29.0000  29.0000
10              cor  4106.000000  4106.000000  Default  28.0000  28.0000
29             tube  2383.000000  2383.000000  Default  27.0000  27.0000
28           insert  2236.000000  2236.000000  Default  26.0000  26.0000
..              ...          ...          ...      ...      ...      ...
239             fus   209.528130   210.183111   Topic3  -4.8546   1.2298
184  detoxification   205.709753   206.365468   Topic3  -4.8730   1.2298
166        arterial   193.805053   194.460625   Topic3  -4.9326   1.2296
279         replace   195.633095   196.325461   Topic3  -4.9232   1.2294
214       lobectomy   196.369515   197.152719   Topic3  -4.9194   1.2290

[125 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
176       3  0.996799          agent
126       1  0.992578          anast
21        3  0.998369            ant
9         1  0.999933            art
166       3  0.997631       arterial
...     ...       ...            ...
169       2  0.999339          valve
35        2  0.997750  valvuloplasty
43        1  0.999491         venous
4         2  0.999173           vent
84        2  0.998228         vessel

[96 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])

In [38]:
from gensim.models import LdaModel
topic_model_class = lda_model
ensemble_workers = 4
num_models=8
distance_workers = 4
num_topics=3
passes = 2

In [39]:
from gensim.models import EnsembleLda
ensemble = EnsembleLda(
    corpus=corpus,
    id2word=id2word,
    num_topics=num_topics,
    passes=passes,
    num_models=num_models,
    topic_model_class=LdaModel,
    ensemble_workers=ensemble_workers,
    distance_workers=distance_workers
)

print(len(ensemble.ttda))
print(len(ensemble.get_topics()))


/Users/maximascientia/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/maximascientia/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/maximascientia/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/maximascientia/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/maximascientia/opt/anaconda3/lib/

24
2


In [40]:
import numpy as np
shape = ensemble.asymmetric_distance_matrix.shape
without_diagonal = ensemble.asymmetric_distance_matrix[~np.eye(shape[0], dtype=bool)].reshape(shape[0], -1)
print(without_diagonal.min(), without_diagonal.mean(), without_diagonal.max())

ensemble.recluster(eps=0.09, min_samples=2, min_cores=2)

print(len(ensemble.get_topics()))

0.011190262732312495 0.5089369141739286 1.0
2


/Users/maximascientia/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/maximascientia/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/maximascientia/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/maximascientia/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/maximascientia/opt/anaconda3/lib/

In [41]:
from gensim.models import LdaMulticore

model1 = LdaMulticore(
    corpus=corpus,
    id2word=id2word,
    num_topics=9,
    passes=4,
)

model2 = LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=11,
    passes=2,
)

# add_model supports various types of input, check out its docstring
ensemble.add_model(model1)
ensemble.add_model(model2)

ensemble.recluster()

print(len(ensemble.ttda))
print(len(ensemble.get_topics()))

44
2
